# Comparaison de LOF, OC-SVM et Isolation Forest

## Jeu de données

Nous utilisons le jeu de données KDD99CUP

## Métriques de comparaison

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from matplotlib.mlab import frange
import statistics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve


## Chargement du jeu de données KDD99-Cup HTTP

KDD99-Cup HTTP contient 30 attributs dont le dernier est la classe à expliquer.
Il y a 620098 observations dont 1052 anormales.
Avec le dernier attribut, nous avons les classes "o" pour les anomalies et "n" pour les observations normales.

In [2]:
data_brut_KDD99CUP = pd.read_csv('/Users/thesard/Doctorat/These2018/ISEP/Developpements/EspaceIntelliJ/LearningAllInPython/Learning_IHM_Features/datasets/imported_datasets/kdd99-unsupervised-ad_2019-06-24 16:08:29.815617.csv', header=None, index_col=None)
X_KDD99CUP = data_brut_KDD99CUP[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13 ,14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]]
y_brut_KDD99CUP = data_brut_KDD99CUP[[29]]
data_brut_KDD99CUP.describe()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
count,620098.000000,620098.000000,6.200980e+05,620098.000000,620098.000000,620098.000000,620098.000000,620098.000000,620098.000000,620098.000000,...,620098.000000,620098.000000,620098.000000,620098.000000,620098.000000,620098.000000,620098.000000,620098.000000,620098.000000,620098.000000
mean,0.627670,280.374673,4.099298e+03,0.005012,0.912428,0.000947,0.000318,0.000013,0.000010,0.005409,...,127.698101,244.485422,0.996744,0.000796,0.074723,0.031820,0.001920,0.001473,0.087651,0.086732
std,53.972528,1509.901935,2.369388e+04,0.098271,0.282671,0.045654,0.017821,0.003592,0.003111,0.073346,...,105.897233,40.458619,0.033137,0.009522,0.185812,0.050689,0.030694,0.028037,0.274777,0.268543
min,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,210.000000,4.580000e+02,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,22.000000,255.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,240.000000,1.425000e+03,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,96.000000,255.000000,1.000000,0.000000,0.010000,0.020000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,302.000000,3.731000e+03,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.000000,0.050000,0.040000,0.000000,0.000000,0.000000,0.000000
max,41065.000000,54540.000000,1.173059e+07,21.000000,1.000000,21.000000,1.000000,1.000000,1.000000,1.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
# Nombre d'anomalies
n_outliers = data_brut_KDD99CUP[data_brut_KDD99CUP[29] == 'o']
len(n_outliers)/6

175.33333333333334

In [26]:
#Pourcentage d'anomalies
pourcentage_anomalies = len(n_outliers)/len(data_brut_KDD99CUP)
pourcentage_anomalies

0.0016965060361426743

In [15]:
# Nombre de données normales
n_normals = data_brut_KDD99CUP[data_brut_KDD99CUP[29] == 'n']
len(n_normals)/6

103174.33333333333

In [12]:
#Pourcentage de données normales
pourcentage_normales = len(n_normals)/len(data_brut_KDD99CUP)
pourcentage_normales

0.9983034939638573

## Spliter le jeu de donner pour réduire la taille
Ceci sera fait en tenant compte du taux d'anomalies dans le jeu de données principale.
On aura au final 6 sous jeux de données sur lesquels chaque méthode sera appliquée. 
La moyenne des résultats sera alors le résultat à retenir pour tout le jeu de données.
NB: 
    - CPU Time = Somme
    - Précion  = Moyenne ou Somme des différentes matrices de confusion
    - Rappel   = Moyenne ou Somme des différentes matrices de confusion
    - ROC AUC  = Moyenne
    - f1-score = Moyenne
    - TNBC     = Moyenne ou Somme des différentes matrices de confusion
    - TPBC     = Moyenne ou Somme des différentes matrices de confusion

In [27]:
# Spliter à l'aide Stratifield mais ça ne donne pas le résultat que je veux. ça donne 6 jeux d'entrainement et
# et de test, tous de même taille dont la somme des jeux de test et d'entrainement donne la taille du jeu de 
# données original.
#from sklearn.model_selection import StratifiedShuffleSplit
#split = StratifiedShuffleSplit(n_splits=6, test_size=0.16965060361426743, random_state=42)
#i = 0
#for train_index, test_index in split.split(data_brut_KDD99CUP, data_brut_KDD99CUP[29]):
#    i=i+1
#    strat_train_set = data_brut_KDD99CUP.loc[train_index]
#    print("train :"+str(i))
#    #print(strat_train_set)
#    print("Taille ="+str(len(strat_train_set)))
#    strat_train_outliers = strat_train_set[strat_train_set[29] == 'o']
#    strat_train_normals  = strat_train_set[strat_train_set[29] == 'n']
#    print("n_outliers ="+str(len(strat_train_outliers)/(len(strat_train_outliers) + len(strat_train_normals))))
#    print("%_outliers ="+str(len(strat_train_outliers)))
#    print("n_normals  ="+str(len(strat_train_normals)))
#    print("%_normals  ="+str(len(strat_train_normals)/(len(strat_train_outliers) + len(strat_train_normals))))
#    strat_test_set = data_brut_KDD99CUP.loc[test_index]
#    print("test :"+str(i))
#    #print(strat_test_set)
#    print("Taille ="+str(len(strat_test_set)))
#    strat_test_outliers = strat_test_set[strat_test_set[29] == 'o']
#    strat_test_normals  = strat_test_set[strat_test_set[29] == 'n']
#    print("n_outliers ="+str(len(strat_test_outliers)))
#    print("%_outliers ="+str(len(strat_test_outliers)/(len(strat_test_outliers) + len(strat_test_normals))))
#    print("n_normals  ="+str(len(strat_test_normals)))
#    print("%_normals  ="+str(len(strat_test_normals)/(len(strat_test_outliers) + len(strat_test_normals))))
#    print("\n")

train :1
Taille =514898
n_outliers =0.001697423567386162
%_outliers =874
n_normals  =514024
%_normals  =0.9983025764326139
test :1
Taille =105200
n_outliers =178
%_outliers =0.0016920152091254752
n_normals  =105022
%_normals  =0.9983079847908746


train :2
Taille =514898
n_outliers =0.001697423567386162
%_outliers =874
n_normals  =514024
%_normals  =0.9983025764326139
test :2
Taille =105200
n_outliers =178
%_outliers =0.0016920152091254752
n_normals  =105022
%_normals  =0.9983079847908746


train :3
Taille =514898
n_outliers =0.001697423567386162
%_outliers =874
n_normals  =514024
%_normals  =0.9983025764326139
test :3
Taille =105200
n_outliers =178
%_outliers =0.0016920152091254752
n_normals  =105022
%_normals  =0.9983079847908746


train :4
Taille =514898
n_outliers =0.001697423567386162
%_outliers =874
n_normals  =514024
%_normals  =0.9983025764326139
test :4
Taille =105200
n_outliers =178
%_outliers =0.0016920152091254752
n_normals  =105022
%_normals  =0.9983079847908746


train :5

In [31]:
# Subset the dataset by myself
import math
n_i_min = 0
o_i_min = 0
n_i_max = math.ceil(len(n_normals)/6)
o_i_max = math.ceil(len(n_outliers)/6)
n_sub_dataset_1 = n_normals[n_i_min:n_i_max]
print("Length normals subset 1 = "+str(len(n_sub_dataset_1)))
o_sub_dataset_1 = n_outliers[o_i_min:o_i_max]
print("Length outliers subset 1 = "+str(len(o_sub_dataset_1)))

n_i_min = n_i_max
o_i_min = o_i_max
n_i_max = n_i_max + math.ceil(len(n_normals)/6)
o_i_max = o_i_max + math.ceil(len(n_outliers)/6)
n_sub_dataset_2 = n_normals[n_i_min:n_i_max]
print("Length normals subset 2 = "+str(len(n_sub_dataset_2)))
o_sub_dataset_2 = n_outliers[o_i_min:o_i_max]
print("Length outliers subset 2 = "+str(len(o_sub_dataset_2)))

n_i_min = n_i_max
o_i_min = o_i_max
n_i_max = n_i_max + math.ceil(len(n_normals)/6)
o_i_max = o_i_max + math.ceil(len(n_outliers)/6)
n_sub_dataset_3 = n_normals[n_i_min:n_i_max]
print("Length normals subset 3 = "+str(len(n_sub_dataset_3)))
o_sub_dataset_3 = n_outliers[o_i_min:o_i_max]
print("Length outliers subset 3 = "+str(len(o_sub_dataset_3)))

n_i_min = n_i_max
o_i_min = o_i_max
n_i_max = n_i_max + math.ceil(len(n_normals)/6)
o_i_max = o_i_max + math.ceil(len(n_outliers)/6)
n_sub_dataset_4 = n_normals[n_i_min:n_i_max]
print("Length normals subset 4 = "+str(len(n_sub_dataset_4)))
o_sub_dataset_4 = n_outliers[o_i_min:o_i_max]
print("Length outliers subset 4 = "+str(len(o_sub_dataset_4)))

n_i_min = n_i_max
o_i_min = o_i_max
n_i_max = n_i_max + math.ceil(len(n_normals)/6)
o_i_max = o_i_max + math.ceil(len(n_outliers)/6)
n_sub_dataset_5 = n_normals[n_i_min:n_i_max]
print("Length normals subset 5 = "+str(len(n_sub_dataset_5)))
o_sub_dataset_5 = n_outliers[o_i_min:o_i_max]
print("Length outliers subset 5 = "+str(len(o_sub_dataset_5)))

n_i_min = n_i_max
o_i_min = o_i_max
n_i_max = n_i_max + math.ceil(len(n_normals)/6)
o_i_max = o_i_max + math.ceil(len(n_outliers)/6)
n_sub_dataset_6 = n_normals[n_i_min:n_i_max]
print("Length normals subset 6 = "+str(len(n_sub_dataset_6)))
o_sub_dataset_6 = n_outliers[o_i_min:o_i_max]
print("Length outliers subset 6 = "+str(len(o_sub_dataset_6)))

Length normals subset 1 = 103175
Length outliers subset 1 = 176
Length normals subset 2 = 103175
Length outliers subset 2 = 176
Length normals subset 3 = 103175
Length outliers subset 3 = 176
Length normals subset 4 = 103175
Length outliers subset 4 = 176
Length normals subset 5 = 103175
Length outliers subset 5 = 176
Length normals subset 6 = 103171
Length outliers subset 6 = 172


## Transformation de la classe à expliquer

L'objectif ici est de remplacer les "o" par -1 et les "n" par 1 afin de faire les matrices de confusion avec la fonction dédiée de scikit-learn.

In [3]:
y_transform_KDD99CUP = y_brut_KDD99CUP
y_transform_KDD99CUP = np.where(y_transform_KDD99CUP=='o',-1,1)
#y_transform_KDD99CUP

## KDD99CUP

### Exécution de Isolation Forest sur le jeux de données KDD99CUP.

In [ ]:

max_samples = 'auto'

averages_executions_time_IF_KDD99CUP = []
averages_roc_auc_IF_KDD99CUP = []

for j in range(50, 300, 10):
    n_estimators = j
    executions_time_IF_KDD99CUP = []
    roc_auc_IF_KDD99CUP = []
    for i in range(5):
        start_IF_KDD99CUP = time()
        
        func_IF_KDD99CUP = IsolationForest(n_estimators=n_estimators, max_samples=max_samples)
        func_IF_KDD99CUP.fit(X_KDD99CUP)
        y_pred_IF_KDD99CUP = func_IF_KDD99CUP.predict(X_KDD99CUP)

        # Calcul du temps d'exécution
        exec_time_IF_KDD99CUP = time() - start_IF_KDD99CUP
        executions_time_IF_KDD99CUP.append(exec_time_IF_KDD99CUP)

        # Calcul de l'aire sous la courbe ROC
        auc_IF_KDD99CUP = roc_auc_score(y_transform_KDD99CUP, y_pred_IF_KDD99CUP)
        roc_auc_IF_KDD99CUP.append(auc_IF_KDD99CUP)
        #print("ROC AUC for Isolation Forest applied on KDD99CUP")
        #print(roc_auc_IF_KDD99CUP)
    print("Execution time of IsolationForest applied on KDD99CUP (in second)")
    print(executions_time_IF_KDD99CUP)
    print("ROC AUC for Isolation Forest applied on KDD99CUP")
    print(roc_auc_IF_KDD99CUP)
    averages_executions_time_IF_KDD99CUP.append(statistics.mean(executions_time_IF_KDD99CUP))
    averages_roc_auc_IF_KDD99CUP.append(statistics.mean(roc_auc_IF_KDD99CUP))

/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:213: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:223: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:213: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarni

Execution time of IsolationForest applied on KDD99CUP (in second)
[28.578444957733154, 25.23741888999939, 26.253039121627808, 26.557228088378906, 25.413676023483276]
ROC AUC for Isolation Forest applied on KDD99CUP
[0.8450864858915931, 0.7922296777911022, 0.9245794559343361, 0.9031552769858107, 0.7946295267847991]


/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:213: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:223: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:213: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarni

Execution time of IsolationForest applied on KDD99CUP (in second)
[32.12448501586914, 31.418391942977905, 31.097649097442627, 31.948478937149048, 30.568878889083862]
ROC AUC for Isolation Forest applied on KDD99CUP
[0.8817407903703269, 0.8065092329791054, 0.8260282327711195, 0.8507996002778665, 0.9117257562596409]


/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:213: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:223: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:213: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarni

...

/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:213: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:223: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:213: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarni

Execution time of IsolationForest applied on KDD99CUP (in second)
[764.371738910675, 193.20265579223633, 189.09385180473328, 185.24283504486084, 211.38799500465393]
ROC AUC for Isolation Forest applied on KDD99CUP
[0.8907768624822183, 0.8431683836243599, 0.8341218114850067, 0.8298369756953017, 0.8969668774284346]


/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:213: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:223: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:213: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarni

Execution time of IsolationForest applied on KDD99CUP (in second)
[209.19160890579224, 202.19053626060486, 209.57726192474365, 200.8908610343933, 202.2378649711609]
ROC AUC for Isolation Forest applied on KDD99CUP
[0.8974421625995372, 0.8969846467056773, 0.827954411982554, 0.8241238613090283, 0.9270083819271574]


/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:213: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:223: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/Users/thesard/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:213: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarni

### Résultat de Isolation Forest sur KDD99CUP

In [ ]:
print("Execution time of IsolationForest applied on KDD99CUP (in second)")
print(averages_executions_time_IF_KDD99CUP)
plt.plot(range(50, 300, 10), averages_executions_time_IF_KDD99CUP,  "g-", label="IForest on KDD99CUP")
#plt.axis([0, 1, 0, 1])
plt.xlabel('n_estimators')
plt.ylabel('Execution time')
plt.legend(loc="best")
plt.show()

In [ ]:
print("ROC AUC for Isolation Forest applied on KDD99CUP")
print(averages_roc_auc_IF_KDD99CUP)
print("La valeur max est "+str(max(averages_roc_auc_IF_KDD99CUP))+" à l'index "+str(averages_roc_auc_IF_KDD99CUP.index(max(averages_roc_auc_IF_KDD99CUP))))
plt.plot(range(50, 300, 10), averages_roc_auc_IF_KDD99CUP,  "g-", label="ROC AUC of IForest on KDD99CUP")
#plt.axis([0, 1, 0, 1])
plt.xlabel('n_estimators')
plt.ylabel('AUC')
plt.legend(loc="best")
plt.show()

### Interprétation du résultat de IsolationForest sur KDD99CUP

Le temps d'exécution de IForest n'est pas lié à la variation de contamination (le taux d'anomalies dans le jeu de données).
Par contre en ce qui concerne l'aire sous la coube ROC, la meilleure performance est arrivée quand contamination est à 0,1. 0,1 est d'ailleurs la valeur par défaut de contamination au niveau de scikit-learn.
Conclusion: Refaire l'exercice en faisant varier cette fois-ci n_estimators (le nombre d'estimateurs de l'emsemble à utiliser pour tester l'aberrance des observations).

Avec n_estimators entre 50 et 300, on remarque que le temps d'exécution augmente au fur et à mesure que le nombre d'estimateurs augmente. Concernant la performance, la meilleure estimation a été obtenu pour un taux de AUC = 0.9530882878060613 avec n_estimators = 70 ===> (50+2*10)
A noter que quelque soit la valeur de n_estimators, la AUC est toujours supérieure à 0,95

### Exécution de LOF sur KDD99CUP

In [ ]:
averages_executions_time_LOF_KDD99CUP = []
averages_roc_auc_LOF_KDD99CUP = []

for j in range(5, 200, 5):
    n_neighbors = j
    executions_time_LOF_KDD99CUP = []
    roc_auc_LOF_KDD99CUP = []
    for i in range(5):
        start_LOF_KDD99CUP = time()
        
        func_LOF_KDD99CUP = LocalOutlierFactor(n_neighbors=n_neighbors,novelty=True)
        func_LOF_KDD99CUP.fit(X_KDD99CUP)
        y_pred_LOF_KDD99CUP = func_LOF_KDD99CUP.predict(X_KDD99CUP)

        # Calcul du temps d'exécution
        exec_time_LOF_KDD99CUP = time() - start_LOF_KDD99CUP
        executions_time_LOF_KDD99CUP.append(exec_time_LOF_KDD99CUP)

        # Calcul de l'aire sous la courbe ROC
        auc_LOF_KDD99CUP = roc_auc_score(y_transform_KDD99CUP, y_pred_LOF_KDD99CUP)
        roc_auc_LOF_KDD99CUP.append(auc_LOF_KDD99CUP)
        
    print("Execution time of LOF applied on KDD99CUP (in second)")
    print(executions_time_LOF_KDD99CUP)
    print("ROC AUC for LOF applied on KDD99CUP")
    print(roc_auc_LOF_KDD99CUP)
    averages_executions_time_LOF_KDD99CUP.append(statistics.mean(executions_time_LOF_KDD99CUP))
    averages_roc_auc_LOF_KDD99CUP.append(statistics.mean(roc_auc_LOF_KDD99CUP))


### Résultat de LOF sur KDD99CUP

In [ ]:
print("Execution time of LOF applied on KDD99CUP (in second)")
print(averages_executions_time_LOF_KDD99CUP)
plt.plot(range(5, 200, 5), averages_executions_time_LOF_KDD99CUP,  "g-", label="LOF on KDD99CUP")
#plt.axis([0, 200, 0, 200])
plt.xlabel('k')
plt.ylabel('Execution time')
plt.legend(loc="best")
plt.show()

In [ ]:
print("ROC AUC for LOF applied on KDD99CUP")
print(averages_roc_auc_LOF_KDD99CUP)
print("La valeur max est "+str(max(averages_roc_auc_LOF_KDD99CUP))+" à l'index "+str(averages_roc_auc_LOF_KDD99CUP.index(max(averages_roc_auc_LOF_KDD99CUP))))
plt.plot(range(5, 200, 5), averages_roc_auc_LOF_KDD99CUP,  "g-", label="ROC AUC of LOF on KDD99CUP")
#plt.axis([0, 1, 0, 1])
plt.xlabel('k')
plt.ylabel('AUC')
plt.legend(loc="best")
plt.show()

### Interprétation du résultat de LOF sur KDD99CUP

Plus le nombre de n_neighbors augmente, plus de temps d'éxécution est important. Ceci s'explique par le fait que plus le nombre de voisins à considérer est grand plus il faudra calculer les distances et les densité afin d'estimer le degré d'abberrance d'une observation.
Concernant la performance de la méthode, elle atteint son plus fort (AUC = 0.6455513956598792) à taux à n_estimators= 80 ((15+1)*5) pour un temps d'exécution de 22.768262100219726

### Exécution de OC-SVM sur KDD99CUP

In [ ]:
averages_executions_time_OCSVM_KDD99CUP = []
averages_roc_auc_OCSVM_KDD99CUP = []

for j in frange(0.1, 1., 0.1):
    nu = j
    executions_time_OCSVM_KDD99CUP = []
    roc_auc_OCSVM_KDD99CUP = []
    for i in range(5):
        start_OCSVM_KDD99CUP = time()
        
        func_OCSVM_KDD99CUP = OneClassSVM(nu=nu)
        func_OCSVM_KDD99CUP.fit(X_KDD99CUP)
        y_pred_OCSVM_KDD99CUP = func_OCSVM_KDD99CUP.predict(X_KDD99CUP)

        # Calcul du temps d'exécution
        exec_time_OCSVM_KDD99CUP = time() - start_OCSVM_KDD99CUP
        executions_time_OCSVM_KDD99CUP.append(exec_time_OCSVM_KDD99CUP)

        # Calcul de l'aire sous la courbe ROC
        auc_OCSVM_KDD99CUP = roc_auc_score(y_transform_KDD99CUP, y_pred_OCSVM_KDD99CUP)
        roc_auc_OCSVM_KDD99CUP.append(auc_OCSVM_KDD99CUP)
        
    print("Execution time of OC-SVM applied on KDD99CUP (in second)")
    print(executions_time_OCSVM_KDD99CUP)
    print("ROC AUC for OC-SVM applied on KDD99CUP")
    print(roc_auc_OCSVM_KDD99CUP)
    averages_executions_time_OCSVM_KDD99CUP.append(statistics.mean(executions_time_OCSVM_KDD99CUP))
    averages_roc_auc_OCSVM_KDD99CUP.append(statistics.mean(roc_auc_OCSVM_KDD99CUP))

### Résultat de OC-SVM sur KDD99CUP

In [ ]:
print("Execution time of OC-SVM applied on KDD99CUP (in second)")
print(averages_executions_time_OCSVM_KDD99CUP)
plt.plot(frange(0.1, 1., 0.1), averages_executions_time_OCSVM_KDD99CUP,  "g-", label="OC-SVM on KDD99CUP")
#plt.axis([0, 1, 0, 1])
plt.xlabel('nu')
plt.ylabel('Execution time')
plt.legend(loc="best")
plt.show()

In [ ]:
print("ROC AUC for OC-SVM applied on KDD99CUP")
print(averages_roc_auc_OCSVM_KDD99CUP)
plt.plot(frange(0.1, 1., 0.1), averages_roc_auc_OCSVM_KDD99CUP,  "g-", label="ROC AUC of OCSVM on KDD99CUP")
#plt.axis([0, 1, 0, 1])
plt.xlabel('nu')
plt.ylabel('AUC')
plt.legend(loc="best")
plt.show()

### Interprétation du résultat de OC-SVM sur KDD99CUP

Plus la valeur de nu augmente plus le temps d'exécution diminue. OC-SVM est donc plus rapide sur KDD99CUP avec un nu plus grand.
Mais, celà n'entraîne pas un bon résultat de la méthode. Puisqu'à partir d'un nu de 0.5, la prédiction de l'algorithme devient de plus en plus aléatoire. La meilleure performance est atteinte à nu= 0.4

## Exécution des méthodes avec les meilleurs paramètres trouvés sur KDD99CUP

In [ ]:
executions_time_KDD99CUP = []
precisions_KDD99CUP = []
recalls_KDD99CUP = []
roc_aucs_KDD99CUP = []
f1_scores_KDD99CUP = []
methods = []

# Isolation Forest
methods.append('IForest')
start_time = time()
n_estimators = 70
func_IF_KDD99CUP = IsolationForest(n_estimators=n_estimators, max_samples=max_samples)
func_IF_KDD99CUP.fit(X_KDD99CUP)
y_pred_IF_KDD99CUP = func_IF_KDD99CUP.predict(X_KDD99CUP)
# Calcul du temps d'exécution
executions_time_KDD99CUP.append(time() - start_time)
# Precision
precisions_KDD99CUP.append(precision_score(y_transform_KDD99CUP, y_pred_IF_KDD99CUP))
# Recall
recalls_KDD99CUP.append(recall_score(y_transform_KDD99CUP, y_pred_IF_KDD99CUP))
# f1_score
f1_scores_KDD99CUP.append(f1_score(y_transform_KDD99CUP, y_pred_IF_KDD99CUP))
# Calcul de l'aire sous la courbe ROC
auc_IF_KDD99CUP = roc_auc_score(y_transform_KDD99CUP, y_pred_IF_KDD99CUP)
roc_aucs_KDD99CUP.append(auc_IF_KDD99CUP)
print("Confusion Matrice IForest")
print(confusion_matrix(y_transform_KDD99CUP, y_pred_IF_KDD99CUP))

# LOF
methods.append('LOF')
start_time = time()
n_neighbors = 80
func_LOF_KDD99CUP = LocalOutlierFactor(n_neighbors=n_neighbors,novelty=True)
func_LOF_KDD99CUP.fit(X_KDD99CUP)
y_pred_LOF_KDD99CUP = func_LOF_KDD99CUP.predict(X_KDD99CUP)
# Calcul du temps d'exécution
executions_time_KDD99CUP.append(time() - start_time)
# Precision
precisions_KDD99CUP.append(precision_score(y_transform_KDD99CUP, y_pred_LOF_KDD99CUP))
# Recall
recalls_KDD99CUP.append(recall_score(y_transform_KDD99CUP, y_pred_LOF_KDD99CUP))
# f1_score
f1_scores_KDD99CUP.append(f1_score(y_transform_KDD99CUP, y_pred_LOF_KDD99CUP))
# Calcul de l'aire sous la courbe ROC
auc_LOF_KDD99CUP = roc_auc_score(y_transform_KDD99CUP, y_pred_LOF_KDD99CUP)
roc_aucs_KDD99CUP.append(auc_LOF_KDD99CUP)
print("Confusion Matrice LOF")
print(confusion_matrix(y_transform_KDD99CUP, y_pred_LOF_KDD99CUP))

# OC-SVM
methods.append('OC-SVM')
start_time = time()
nu = 0.4
func_OCSVM_KDD99CUP = OneClassSVM(nu=nu)
func_OCSVM_KDD99CUP.fit(X_KDD99CUP)
y_pred_OCSVM_KDD99CUP = func_OCSVM_KDD99CUP.predict(X_KDD99CUP)
# Calcul du temps d'exécution
executions_time_KDD99CUP.append(time() - start_time)
# Precision
precisions_KDD99CUP.append(precision_score(y_transform_KDD99CUP, y_pred_OCSVM_KDD99CUP))
# Recall
recalls_KDD99CUP.append(recall_score(y_transform_KDD99CUP, y_pred_OCSVM_KDD99CUP))
# f1_score
f1_scores_KDD99CUP.append(f1_score(y_transform_KDD99CUP, y_pred_OCSVM_KDD99CUP))
# Calcul de l'aire sous la courbe ROC
auc_OCSVM_KDD99CUP = roc_auc_score(y_transform_KDD99CUP, y_pred_OCSVM_KDD99CUP)
roc_aucs_KDD99CUP.append(auc_OCSVM_KDD99CUP)
print("Confusion Matrice OC-SVM")
print(confusion_matrix(y_transform_KDD99CUP, y_pred_OCSVM_KDD99CUP))

## Présentation des résultats sur KDD99CUP

### Résultat : Temps d'exécution, précision, rappel, score f1, AUC

In [ ]:
print("Methods ===>")
print(methods)
print("Execution Time ===>")
print(executions_time_KDD99CUP)
print("Precision ===>")
print(precisions_KDD99CUP)
print("Recall ===>")
print(recalls_KDD99CUP)
print("f1_score ===>")
print(f1_scores_KDD99CUP)
print("ROC AUC ===>")
print(roc_aucs_KDD99CUP)

### Interprétation des résultats en fonction du Temps d'exécution, précision, rappel, score f1, AUC sur KDD99CUP
IsolationForest a eu de meilleures performances que les autres.
Entre OC-SVM et LOF : OC-SVM a découvert et bien classé plus d'anomalies que LOF. Par contre, LOF a découvert et bien placé plus d'observations normales que OC-SVM.
La question est donc de savoir ce qui importe le plus entre la découverte des anomalies et celle des observations normales car c'est en fonction de l'importance que nous donnons à l'un ou à l'autre que nous pourront dire la méthode la meilleure entre OC-SVM et LOF.
Cette question fait remarquer qu'en réalité, toutes les méthodes ont sont performantes dans des conditions données en fonction de ce que l'on souhaite obtenir comme résultat. Dans ce cas précis, OC-SVM lance beaucoup de fausse alertes mais attrappe beaucoup d'anomalies. Par contre, LOF n'attrappe pas beaucoup d'anomalies mais reconnais beaucoup de données normales.
On peut peut-être se dire que dans le cas espèce OC-SVM est mieux dans la mesure où une méthode de détection d'anomalies a pour but principal de détecter les anomalies et donc laisser passer des anomalies en ferait une méthode moins bonne et que les observations normales que OC-SVM aurait classé comme anormales sont peut être des nouveautés.
Considérant l'aire sous la courbe ROC, nous avons l'ordre suivant IForest - OC-SVM et LOF

### Resultat: Courbe ROC pour KDD99CUP

In [ ]:
fpr_IF_KDD99CUP, tpr_IF_KDD99CUP, thresholds_IF_KDD99CUP = roc_curve(y_transform_KDD99CUP, y_pred_IF_KDD99CUP)
fpr_LOF_KDD99CUP, tpr_LOF_KDD99CUP, thresholds_LOF_KDD99CUP = roc_curve(y_transform_KDD99CUP, y_pred_LOF_KDD99CUP)
fpr_OCSVM_KDD99CUP, tpr_OCSVM_KDD99CUP, thresholds_OCSVM_KDD99CUP = roc_curve(y_transform_KDD99CUP, y_pred_OCSVM_KDD99CUP)

In [ ]:
plt.plot(fpr_IF_KDD99CUP, tpr_IF_KDD99CUP, "g-", label="IForest") 
plt.plot(fpr_LOF_KDD99CUP, tpr_LOF_KDD99CUP, "b--", label="LOF") 
plt.plot(fpr_OCSVM_KDD99CUP, tpr_OCSVM_KDD99CUP, "r-", label="OC-SVM") 
plt.plot([0, 1], [0, 1], 'k--') 
plt.axis([0, 1, 0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
    
plt.show()

### Interprétation de la courbe ROC
La courbe ROC mesurant le taux de faux positif par rapport au taux de vrai positif, plus la courbe s'éloigne de celle oblique en noir, mieux c'est.
Ainsi, on peut dire que pour KDD99CUP, IForest est la meilleure méthode de détection d'anomalies suivi par OC-SVM. Le classement de LOF pourrait s'expliquer par le fait que les anomalies qui y sont présentes sont des anomalies global alors que LOF est doué pour les anomalies locales. POur vérifier ça peut être qu'on peut utiliser une méthode de clustering comme K-means pour voir comment les anomalies seront classées par cette méthode.

## Conclusion KDD99CUP
Pour un petit jeu de données statique de faible dimension, Isolation forest a tendance à mieux performer et semble être plus rapide que OC-SVM et LOF. OC-SVM détecte beaucoup d'anomalies avec un fort taux de fausses alertes et un temps d'exécution plus grand que LOF. Par contre, LOF détecte moins d'anomalies avec un faible taux de fausses alertes.
Notons que la performance des méthodes dépends non seulement du jeu de données considéré mais aussi des valeurs des hypermaramètres. Pour LOF et IsolationForest, le temps d'exécution semble être en correlation positive avec respectivement le nombre de voisins et nombre de d'estimateurs. Par contre, le temps d'exécution serait en correlation négative avec nu.